In [1]:
import pandas as pd
import sqlite3
import kagglehub

# Download latest version
path = kagglehub.dataset_download("terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database")

print("Path to dataset files:", path)

Path to dataset files: /Users/alejandrodiazalvarado/.cache/kagglehub/datasets/terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database/versions/1


In [2]:
db_path = path + "/olist.sqlite"
db_connection = sqlite3.connect(db_path)

In [3]:
print(db_connection.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())

[('product_category_name_translation',), ('sellers',), ('customers',), ('geolocation',), ('order_items',), ('order_payments',), ('order_reviews',), ('orders',), ('products',), ('leads_qualified',), ('leads_closed',)]


In [4]:
pd.read_sql_query("SELECT * FROM customers", db_connection)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [7]:
#¿Cuántas órdenes se han realizado en total?
pd.read_sql_query(
    """
    SELECT COUNT(*) AS Total FROM orders
    """,
    db_connection
)

,Total
0,99441


In [9]:
#¿Cuál es el valor total de las órdenes?
pd.read_sql_query(
    """
    SELECT SUM(payment_value) AS Sum FROM order_payments
    """,
    db_connection
)

,Sum
0,16008872.12


In [11]:
#¿Cuál es el producto más vendido por cantidad de veces que aparece en order_items?
pd.read_sql_query(
    """
    SELECT oi.product_id, product_category_name, COUNT(*) AS count FROM order_items oi 
    JOIN products p ON oi.product_id = p.product_id 
    GROUP BY oi.product_id 
    ORDER BY count DESC 
    LIMIT 5
    """,
    db_connection
)

,product_id,product_category_name,count
0,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,527
1,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,488
2,422879e10f46682990de24d770e7f83d,ferramentas_jardim,484
3,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,392
4,368c6c730842d78016ad823897a372db,ferramentas_jardim,388


In [12]:
#¿Cuál es el producto con mayor ingreso total?
pd.read_sql_query(
    """
    SELECT oi.product_id, product_category_name, COUNT(*) AS count, oi.price, COUNT(*) * oi.price AS total FROM order_items oi 
    JOIN products p ON oi.product_id = p.product_id 
    GROUP BY oi.product_id 
    ORDER BY total DESC 
    LIMIT 5
    """,
    db_connection
)

,product_id,product_category_name,count,price,total
0,bb50f2e236e5eea0100680137654686c,beleza_saude,195,330.00,64350.00
1,6cdd53843498f92890544667809f1595,beleza_saude,156,349.90,54584.40
2,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,343,149.00,51107.00
3,d6160fb7873f184099d9bc95e30376af,pcs,35,1300.00,45500.00
4,25c38557cf793876c5abdd5931f922db,bebes,38,1106.99,42065.62


In [13]:
#¿Cuál es el estado con más clientes?
pd.read_sql_query(
    """
    SELECT COUNT(DISTINCT customer_unique_id) AS Customers, customer_state FROM customers 
    GROUP BY customer_state 
    ORDER BY Customers DESC 
    LIMIT 10
    """,
    db_connection
)

,Customers,customer_state
0,40302,SP
1,12384,RJ
2,11259,MG
3,5277,RS
4,4882,PR
5,3534,SC
6,3277,BA
7,2075,DF
8,1964,ES
9,1952,GO


In [14]:
#¿Qué categorías tienen el puntaje de reseña arriba del promedio?
pd.read_sql_query(
    """
    WITH avg_review AS (
    SELECT AVG(review_score) AS avg_score FROM order_reviews
    )
    SELECT DISTINCT p.product_category_name, orw.review_score
    FROM order_items oi
    JOIN order_reviews orw ON oi.order_id = orw.order_id
    JOIN products p ON oi.product_id = p.product_id
    JOIN avg_review ar ON 1=1
    WHERE orw.review_score > ar.avg_score;
    """,
    db_connection
)

,product_category_name,review_score
0,cool_stuff,5
1,moveis_decoracao,5
2,ferramentas_jardim,5
3,beleza_saude,5
4,fashion_bolsas_e_acessorios,5
...,...,...
68,la_cuisine,5
69,pc_gamer,5
70,casa_conforto_2,5
71,fashion_roupa_feminina,5


In [15]:
#¿Cuál es el valor total de pagos recibidos por tipo de pago?
pd.read_sql_query(
    """
    SELECT payment_type, SUM(payment_value) AS Sum FROM order_payments
    GROUP BY payment_type
    """,
    db_connection
)

,payment_type,Sum
0,boleto,2869361.27
1,credit_card,12542084.19
2,debit_card,217989.79
3,not_defined,0.00
4,voucher,379436.87


In [16]:
#¿Qué clientes han realizado la mayor cantidad de pedidos (top 3)?
pd.read_sql_query(
    """
    SELECT customer_id, COUNT(order_id) AS count FROM orders
    GROUP BY customer_id
    ORDER BY count DESC
    LIMIT 3
    """,
    db_connection
)

,customer_id,count
0,ffffe8b65bbe3087b653a978c870db99,1
1,ffffa3172527f765de70084a7e53aae8,1
2,ffff42319e9b2d713724ae527742af25,1


In [17]:
#¿Cuál es el promedio de entrega de los pedidos?
pd.read_sql_query(
    """
    SELECT AVG(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp)) AS avg_delivery_time
    FROM orders
    WHERE order_status = 'delivered'
    """,
    db_connection
)

,avg_delivery_time
0,12.558217


In [18]:
#¿Qué vendedores tienen los ingresos mensuales más altos (top 5)?
pd.read_sql_query(
    """
    SELECT seller_id, COUNT(order_id) AS orders_per_seller, SUM(price) AS total FROM order_items
    GROUP BY seller_id
    ORDER BY total DESC
    LIMIT 5
    """,
    db_connection
)

,seller_id,orders_per_seller,total
0,4869f7a5dfa277a7dca6462dcf3b52b2,1156,229472.63
1,53243585a1d6dc2643021fd1853d8905,410,222776.05
2,4a3ca9315b744ce9f8e9374361493884,1987,200472.92
3,fa1c13f2614d7b5c4749cbc52fecda94,586,194042.03
4,7c67e1448b00f6e969d365cea6b010ab,1364,187923.89


In [19]:
#¿Qué estados tienen más clientes?
pd.read_sql_query(
    """
    SELECT COUNT(DISTINCT customer_id) AS Customers, customer_state
    FROM customers
    GROUP BY customer_state
    ORDER BY Customers DESC
    LIMIT 10
    """,
    db_connection
)

,Customers,customer_state
0,41746,SP
1,12852,RJ
2,11635,MG
3,5466,RS
4,5045,PR
5,3637,SC
6,3380,BA
7,2140,DF
8,2033,ES
9,2020,GO


In [20]:
#¿Qué estados tienen más sellers?
pd.read_sql_query(
    """
    SELECT COUNT(DISTINCT seller_id) AS Sellers, seller_state
    FROM sellers
    GROUP BY seller_state
    ORDER BY Sellers DESC
    LIMIT 10
    """,
    db_connection
)

,Sellers,seller_state
0,1849,SP
1,349,PR
2,244,MG
3,190,SC
4,171,RJ
5,129,RS
6,40,GO
7,30,DF
8,23,ES
9,19,BA


In [21]:
#¿Cuales son los 3 vendedores con más productos vendidos?
pd.read_sql_query(
    """
    SELECT seller_id, COUNT(product_id) AS sold_products FROM order_items
    GROUP BY seller_id
    ORDER BY sold_products DESC
    LIMIT 3    
    """,
    db_connection
)

,seller_id,sold_products
0,6560211a19b47992c3666cc44a7e94c0,2033
1,4a3ca9315b744ce9f8e9374361493884,1987
2,1f50f920176fa81dab994f9023523100,1931


In [22]:
db_connection.close()